<h1>Imports</h1>

In [ ]:
import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


<h1>Logging</h1>

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)

<h1>Function Declarations</h1>

In [ ]:
def load_data(data):
    try:
        df = pd.read_csv(data)
    except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
        logging.error(f"Error: {str(e)}.")
        exit(1)
    return df

In [ ]:
def identify_data_types(df, unique_value_threshold=10):
    numerical_columns = []
    categorical_columns = []
    for column in df.columns:
        # If the column is of object type, it's categorical
        if df[column].dtype == 'object':
            categorical_columns.append(column)
        else:
            # If the number of unique values is less than the threshold, consider it categorical
            if df[column].nunique() <= unique_value_threshold:
                categorical_columns.append(column)
            else:
                numerical_columns.append(column)
    return numerical_columns, categorical_columns

In [ ]:
def compute_correlations(data, target_column, top_n=10):
    # Select only numeric columns for correlation
    numeric_data = data.select_dtypes(include=[np.number])
    correlation_with_target = numeric_data.corr()[target_column].sort_values(ascending=False)
    print(f"Top {top_n} Features Correlated with {target_column}:\n")
    print(correlation_with_target.head(top_n))

In [ ]:
def normalize_and_encode(df, numerical_cols, categorical_cols):
    # Define the transformations for numerical and categorical columns
    numerical_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    # Create the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    # Fit and transform the data
    df_transformed = preprocessor.fit_transform(df)
    # Get feature names after one-hot encoding
    one_hot_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
    all_feature_names = numerical_cols + list(one_hot_feature_names)
    # Create a new DataFrame with the transformed data
    df_transformed = pd.DataFrame(df_transformed, columns=all_feature_names)
    return df_transformed

In [ ]:
def prepare_sequence_data(df, patient_identifier_col, target_col, window_col):
    # Sort the dataframe by patient visit identifier and window
    df_sorted = df.sort_values(by=[patient_identifier_col, window_col])
    # Group by patient visit identifier
    grouped = df_sorted.groupby(patient_identifier_col)
    X = []  # To store sequences
    y = []  # To store labels (ICU admission status)
    for _, group in grouped:
        # Find the first instance of ICU admission
        first_icu_admission = group[target_col].cumsum().shift(fill_value=0).eq(1)
        # Exclude data after the first ICU admission
        group = group[~first_icu_admission]
        # Drop columns that are not features (like identifiers and target)
        features = group.drop(columns=[patient_identifier_col, target_col, window_col])
        # Append the sequence of features to X
        X.append(features.values)
        # Append the label (ICU admission status) to y
        y.append(group[target_col].max())  # max() ensures we capture if the patient was ever admitted to ICU
    return X, y

In [ ]:
def create_lstm_cnn_model(input_shape, num_classes, filters=64, kernel_size=3, lstm_units=100, dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    # Convolutional layers
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    # LSTM layer
    model.add(LSTM(lstm_units, activation='tanh'))
    # Flatten the output of the LSTM layers
    model.add(Flatten())
    # Dropout for regularization
    model.add(Dropout(dropout_rate))
    # Dense layer for classification
    model.add(Dense(num_classes, activation='softmax' if num_classes > 1 else 'sigmoid'))
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

    return model

In [ ]:
def main():
    pass

<h1>Script Start</h1>

In [ ]:
if __name__ == "__main__":
    main()